## Sample and fit

Here we generate discrete points from a LIMEPY model and then fit back the model parameters with a downhill simplex method. First we import the model and sampling routine:

In [1]:
from limepy import limepy, sample
from scipy.optimize import fmin
import numpy

We then create a model and then use the sample routine to generate 100 values

In [2]:
W0, rh = 5, 3
k = limepy(W0, 1, M=1e5, rh=rh)

To generate a discrete sample we run sample on the model and specify the number of points

In [3]:
ic = sample(k, N=1e2, verbose=True)

 sample r ...
 sample v ...
   set-up segments for velocity cdf ...
   compute cdf ...
   (rejection) sampling of k^3/2 values ...
   sample angles ...
 convert to cartesian coordinates ...
 done! 
       U: sample =  -5.9760e+06; model =  -5.8345e+06
       K: sample =   2.9552e+06; model =   2.9173e+06
       Q: sample =  -4.9451e-01; model =  -5.0000e-01
  2Kr/Kt: sample =       0.9038; model =       1.0000


All values are in "ic". For example, we can compute the distance to the centre in projection as

In [4]:
R = numpy.sqrt(ic.x**2 + ic.y**2)

Now pretend that this array *R* is our "data" and we want to retrieve the parameters of the underlying model. Lets define a function that returns the (minus) likelihood, a function that we can minimize with (e.g.) a downhill simplex method

In [5]:
def minloglike(par, Rdat):
    # par is an array with parameters: W0 and rh, create a model with parameters par
    m = limepy(par[0], 1, M=1, rh=par[1], project=True) 
    # Return the minus log likelihood, note that the model is normalised to M=1 above
    return -sum(numpy.log(numpy.interp(Rdat, m.R, m.Sigma, right=1e-9)))

Now do the fit and compare the result (it takes a few seconds)

In [6]:
from scipy.optimize import fmin
x0 = [7,6 ] # Starting values
res = fmin(minloglike, x0, args=(R,))
print " \n Result: W0 = %5.3f (True=%5.3f)"%(res[0], W0),"; rh = %5.3f (True=%5.3f)"%(res[1], rh)

Optimization terminated successfully.
         Current function value: 441.401295
         Iterations: 54
         Function evaluations: 120
 
 Result: W0 = 5.336 (True=5.000) ; rh = 2.795 (True=3.000)
